
Задание для личного проекта, который нужно будет сдавать для оценки:
# Написать чат бота, который будет говорить погоду в двух городах - Москва и СПб
# Бот должен уметь здороваться, прощаться.
# Бот должен понимать такие даты, как: сейчас, сегодня, завтра.
# Попрощаться - значит закончить беседу (после прощания, он не должен писать никаких сообщений пока вы не напишите)
# Бот должен как-то реагировать на непонятные высказывания
# Базу погоды можно составить самим - на два дня, на два города или использовать API (например, OpenWeatherMap (https://openweathermap.org/)).
# Бот должен работать и продолжать вас что-то спрашивать пока вы не закончите беседу.
# Бот должен запрашивать всю необходимую информацию для прогноза (если не хватает какого-то слота (город или дата), он должен его уточнить)

Один из сценариев скрипта:
1. Написать функцию распознания интенции (Например, Natasha)
2. Функция для того, чтобы здороваться.
3. Функция для погоды
4. Функция, чтобы прощаться.
5. Функция для ошибок.

То есть, из фич только погода.

Мануал как делать бота на питоне

(https://thecode.media/python-bot/)Если что-то не получается, все падает с ошибкой и т.д. - смело пишите сюда или в личку.

Deadline по проекту: сессионная неделя.
ДЗ можно просто отправить, я посмотрю, и если что-то не так, то доработать.
Проект нужно будет показывать. Можно показать, и если что-то не так, то доделать.

P.S. Каждый член команды должен будет ответить на вопросы по боту и рассказать как это работает.

P.P.S Не забывайте про домашнее задание - это почти половина оценки

In [1]:
import json
import urllib3
import spacy
from secret import *

from datetime import date
from datetime import datetime
import time
nlp = spacy.load('ru_core_news_md')

In [2]:
http = urllib3.PoolManager()

### geo

In [3]:
def place_to_geo(place, city, map_key = map_key):
    '''

    :param place: Name of place to search
    :param city: Name of city
    :param map_key: API KEY from https://developer.mapquest.com/
    :return: dict {'lat': '55.6280353', 'lon': '36.9952076310053'}
    '''
    geo = {}

    url = f'http://open.mapquestapi.com/nominatim/v1/search.php?key={map_key}&format=json&q={place}+{city}&addressdetails=1&limit=3'
    response = http.request("GET", url)
    data = json.loads(response.data)[0]

    geo['lat'] = data['lat']
    geo['lon'] = data['lon']
    return geo

In [4]:
geo_ll =  place_to_geo("Невский проспект", "Москва")
geo_ll

{'lat': '59.9316703', 'lon': '30.3588031'}

### weather

In [5]:
def location_to_weather(geo_ll, weather_key = weather_key):
    '''
    :param geo_ll: dict from place_to_geo
    :param weather_key: api key from https://openweathermap.org/api/one-call-api
    :return: dict with temp for 7 days
    '''
    weather = {}

    lat = geo_ll['lat']
    lon = geo_ll['lon']

    url = f'https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude=current,minutely,hourly,alerts&appid={weather_key}&units=metric'
    response = http.request("GET", url)
    data = json.loads(response.data)
    for day in data['daily']:
        weather[day['dt']]=  day['feels_like']
    return weather


In [6]:
weather = location_to_weather(geo_ll)
weather

{1640077200: {'day': -17.17, 'night': -19.12, 'eve': -18.99, 'morn': -17.47},
 1640163600: {'day': -17.33, 'night': -18.55, 'eve': -18.98, 'morn': -17.46},
 1640250000: {'day': -12.37, 'night': -6.74, 'eve': -7.79, 'morn': -15.96},
 1640336400: {'day': -12.96, 'night': -18.59, 'eve': -9.16, 'morn': -13.19},
 1640422800: {'day': -21.7, 'night': -14.28, 'eve': -15.3, 'morn': -26.92},
 1640509200: {'day': -13.93, 'night': -22.25, 'eve': -22.63, 'morn': -12.19},
 1640595600: {'day': -15.25, 'night': -18.68, 'eve': -17.31, 'morn': -12.94},
 1640682000: {'day': -16.36, 'night': -18.06, 'eve': -19.11, 'morn': -18.73}}

### part of speech

In [7]:
def text_to_location(text, nlp = nlp):
    '''
    :param text: string input
    :param nlp: nlp spacy language model
    :return: list of locations
    '''

    text_spacy = nlp(text)

    locations = []

    if len(text_spacy.ents) == 0:
        raise NameError("Нет локации")
    else:
        for ent in text_spacy.ents:
            if ent.label_ == 'LOC':
                locations.append(ent.lemma_)

    return(locations)

In [8]:
text_to_location("как дела в Саратове и томске")

['саратов', 'томск']

## date

In [22]:
def text_to_date(text, nlp = nlp):
    '''
    :param text: string input
    :param nlp: nlp spacy language model
    :return: relative index of a date
    '''

    today = time.mktime(date.today().timetuple())
    tokens = []
    today_unix = time.mktime(date.today().timetuple())

    text_spacy = nlp(text)
    for token in text_spacy:
        tokens.append(token.lemma_)

    if 'сегодня' in tokens:
        return 0
    elif 'завтра' in tokens:
        return 1
    elif 'сейчас' in tokens:
        return 0
    elif 'послезавтра' in tokens:
        return 2
    else:
        raise NameError('Плохо расслышал дату. Это сегодня, завтра или послезавтра?')

In [23]:
text_to_date("Скажи погоду в Санкт-Петербурге на сегодня")

0

## text to answer

In [13]:
# text  = "Скажи погоду Москве на завтра"
text = "сколько вешать в граммах в Москве"


In [24]:
def message_to_reply(text):
    '''

    :param text: string with text
    :return: reply
    '''
    try:
        location = text_to_location(text)
    except NameError:
        return ("А для какого места тебе нужна погода?")
    try:
        date_index = text_to_date(text)
    except NameError:
        return ("Cкажи, а на какой день делать прогноз?")
    try:
        geo_ll = place_to_geo("", location)
    except NameError:
        return ("Попробуй другую локацию")
    try:
        weather = location_to_weather(geo_ll)
    except NameError:
        return (f'Не смог найти погоду для {location}')
    try:
        weather_day = weather[list(weather.keys())[date_index]]
        day = date.fromtimestamp(list(weather.keys())[date_index])
        return f'Погода в городе {location[0].title()} {day} ' \
               f'днём: {weather_day["day"]} ;' \
               f'вечером {weather_day["eve"]}'
    except NameError:
        return (f' ошибка в последнем действии')


In [25]:
text  = "Скажи погоду Москве на завтра"
# text = "сколько вешать в граммах в Москве"

message_to_reply(text)

'Погода в городе Москва 2021-12-22 днём: -26.61 ;вечером -25.95'